In [1]:
#!pip install --upgrade pip imbalanced-learn
#!pip install tensorflow-gpu==1.14

In [2]:
import pickle
import math
from math import sqrt
import numpy as np
import pandas as pd
import tensorflow as tf
from collections import Counter
import tensorflow.keras as K
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize as scikit_normalize
from sklearn.utils import shuffle as  sklearn_shuffle

from IPython.core.display import display
from IPython.core.display import HTML

%matplotlib inline

### Evaluation functions

In [4]:
DATASET='/home/kvassay/data/z/data/reviews_train_test_dev1_{}.pickle'
TYPE='lem'

## Load data

In [ ]:
%%time
with open(DATASET.format(TYPE),'rb') as f:
    train,dev,test=pickle.load(f)

## Train TF-IDF model

In [ ]:
def tfidf_train(dataset, **scikit_kwargs):
    vectorizer=TfidfVectorizer(**scikit_kwargs)
    vectorizer.fit([x['text'] for x in dataset])
    return vectorizer

def tfidf_predict(vectorizer,dataset, apply_norm=True):
    features=vectorizer.transform([x['text'] for x in dataset])
    if apply_norm is True:
        features=scikit_normalize(features)
    return features

In [ ]:
%%time
vectorizer=tfidf_train(train,max_features=100000,max_df=0.99,ngram_range=(1,2), use_idf=False)

## Extract features

In [ ]:
%%time
X_train=tfidf_predict(vectorizer,train)
X_dev=tfidf_predict(vectorizer,dev)
y_train=np.array([x['score'] for x in train])
y_dev=np.array([x['score'] for x in dev])
print('Train samples shape: {}, Dev samples shape: {}'.format(X_train.shape,X_dev.shape))

## Training

In [ ]:
class UnderSampler(K.utils.Sequence):

    def __init__(self, X, y, batch_size):
        self.rus=RandomUnderSampler(sampling_strategy='not minority')
        self.X, self.y = X, y
        self.batch_size = batch_size
        #len
        self._shuffle()
        self.length= math.ceil(self.X_u.shape[0] / self.batch_size)

    def _shuffle(self):
        self.X_u, self.y_u=self.rus.fit_resample(self.X,self.y)
        self.X_u,self.y_u = sklearn_shuffle(self.X_u,self.y_u)
        
    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        batch_x = self.X_u[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y_u[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x.todense(), batch_y
    
    def on_epoch_end(self):
        self._shuffle()

In [ ]:
def train_model(batch_size=200, epochs=5, sampler_cls=UnderSampler):
    model = K.models.Sequential([
        K.layers.Dense(100,activation='relu', input_shape=(X_train.shape[1],),kernel_initializer='glorot_normal'),
        K.layers.Dense(1,activation='linear',kernel_initializer='glorot_normal'),
    ])
    opt=K.optimizers.Adam(lr=0.02, amsgrad=True)
    model.compile(optimizer=opt,loss='mean_squared_error',metrics=[rmse_partial_avg,rmse_partial_max])
    sampler=sampler_cls(X_train,y_train,batch_size=batch_size)
    model.fit_generator(sampler, shuffle=False, epochs=epochs, validation_data=(X_dev.todense(),y_dev))
    return model

## Evaluate

In [ ]:
def plot_history(model,title):
    loss = model.history.history['loss']
    val_loss = model.history.history['val_loss']
    epochs = range(1, len(loss) + 1)
    plt.plot(epochs, loss, color='red', label='MSE Train')
    plt.plot(epochs, val_loss, color='green', label='MSE Dev')
    plt.title(title)
    plt.xlabel('Epochs') 
    plt.ylabel('MSE')
    plt.legend()
    plt.show()

def rmse(y_true,y_pred):
    return sqrt(mean_squared_error(y_true,y_pred))

def rmse_partial(y_true, y_pred):
    all_scores=list(set(y_true))
    y_true_dict={x:[] for x in all_scores}
    y_pred_dict={x:[] for x in all_scores}
    for i, score in enumerate(y_true):
        y_true_dict[score].append(score)
        y_pred_dict[score].append(y_pred[i])
    return {score: rmse(y_true_dict[score],y_pred_dict[score]) for score in all_scores}
        
def rmse_partial_avg(y_true, y_pred):
    rmse_dict=rmse_partial(y_true,y_pred)
    values=list(rmse_dict.values())
    return np.mean(values), np.std(values)

def rmse_partial_max(y_true, y_pred):
    rmse_dict=rmse_partial(y_true,y_pred)
    values=list(rmse_dict.values())
    return np.max(values)

def rmse_report(y_true,y_pred,round_decimals=3, title='RMSE report'):
    def ar(x):
        return np.around(x,decimals=round_decimals)
    baseline=rmse(y_true,[5.0]*len(y_true))
    HTML_TEMPLATE="""
    <h2> {} </h2>
    <h3> RMSE </h3>
    <hr>
    <div>
        <table>
            <tr>
                <td>RMSE (baseline &forall;1.0)</td>
                <td>{}</td>
            </tr>
            <tr>
                <td>RMSE</td>
                <td>{}</td>
            </tr>
        </table>
    <hr>
    <h3> Partial RMSE </h3>
        <table>
            <tr>
                <td>Mean partial RMSE (baseline &forall;1.0)</td>
                <td>{}</td>
            </tr>
            <tr>
                <td>Max partial RMSE (baseline &forall;1.0)</td>
                <td>{}</td>
            </tr>
            <tr>
                <td>St.dev. partial RMSE (baseline &forall;1.0)</td>
                <td>{}</td>
            </tr>
            <tr>
                <td>Mean partial RMSE</td>
                <td><b>{}</b></td>
            </tr>
            <tr>
                <td>Max partial RMSE</td>
                <td>{}</td>
            </tr>
            <tr>
                <td>St.dev. partial RMSE</td>
                <td>{}</td>
            </tr>            
        </table>
    </div>
    <h3> Improvement over baseline (&forall;1.0) </h3>
    <hr>
    <div>
        <table>
            <tr>
                <td>RMSE</td>
                <td>{}</td>
            </tr>
            <tr>
                <td>Mean partial RMSE</td>
                <td><b>{}</b></td>
            </tr>
            <tr>
                <td>Max partial RMSE</td>
                <td>{}</td>
            </tr>
        </table>
    </div>
    
    <h3> Partial RMSE detailed</h3>
    <hr>
    <div>
        <table>
            <tr>
                <th>Review Score</th>
                <th>RMSE</th>
                <th>RMSE baseline (&forall;1.0)</th>
                <th>Improvement over baseline</th>
            </tr>
            {}
        </table>
    </div>
    """
    PARTIAL_ROW_TEMPLATE='''
    <tr>
        <td>
            {}
        </td>
        <td>
            {}
        </td>
        <td>
            {}
        </td>
        <td>
            {}
        </td>
    </tr>
    '''
    overall=rmse(y_true,y_pred)
    partial_avg,partial_std=rmse_partial_avg(y_true,y_pred)
    partial_max=rmse_partial_max(y_true,y_pred)
    partial_avg_baseline,partial_std_baseline=rmse_partial_avg(y_true,[5.0]*len(y_true))
    partial_max_baseline=rmse_partial_max(y_true,[5.0]*len(y_true))
    improvement_marmse=partial_avg_baseline-partial_avg
    improvement_rmse=baseline-overall
    improvement_rmse_partial_max=partial_max_baseline - partial_max
    
    #partial rows
    partial=rmse_partial(y_true,y_pred)
    partial_baseline=rmse_partial(y_true, [5.0]*len(y_true))
    partial=sorted(partial.items(),key=lambda x:x[0],reverse=True)
    partial_table_rows=[]
    for key,value in partial:
        value_baseline=partial_baseline[key]
        diff_baseline=value_baseline-value
        partial_table_rows.append(PARTIAL_ROW_TEMPLATE.format(key,ar(value),ar(value_baseline),ar(diff_baseline)))
    partial_table_rows='\n'.join(partial_table_rows)
    
    html=HTML_TEMPLATE.format(title,
                              ar(baseline),
                              ar(overall),
                              ar(partial_avg_baseline),
                              ar(partial_std_baseline),
                              ar(partial_max_baseline),
                              ar(partial_avg),
                              ar(partial_std),
                              ar(partial_max),
                              ar(improvement_rmse),
                              ar(improvement_marmse),
                              ar(improvement_rmse_partial_max),
                              partial_table_rows)
    display(HTML(html))

In [ ]:
def experiment(sampling_cls,epochs,batch_size,name):
    model=train_model(sampler_cls=sampling_cls,epochs=epochs,batch_size=batch_size)
    y_pred_dev=model.predict(X_dev)
    rmse_report(y_dev,y_pred_dev,title='{} - RMSE report'.format(name))
    plot_history(model,title='{} - Train/Dev MSE'.format(name))
    return model

In [ ]:
model=experiment(sampling_cls=UnderSampler,epochs=25,batch_size=300,name='TF-IDF model')

In [ ]:
model.summary()

## Persist

In [ ]:
with open('/home/kvassay/data/z/models/E2/vectorizer.pickle','wb') as f:
    pickle.dump(vectorizer,f)
model.save('/home/kvassay/data/z/models/E2/keras_regressor.h5')